### Solid State Physics- Elastic Waves in Cubic crystal

# Wave Propagation in a 3D FCC Lattice

This notebook simulates wave propagation through a 3D face-centered cubic (FCC) lattice structure. By adjusting parameters like wavevector (k), frequency (ω), lattice constant (a), and propagation direction, you can observe the dynamic motion of the lattice atoms and the resulting wave displacements in real-time through interactive 3D visualizations.

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox
from IPython.display import display

# Create FCC lattice points
def create_fcc_lattice(num_cells, lattice_constant):
    positions = []
    for i in range(num_cells):
        for j in range(num_cells):
            for k in range(num_cells):
                positions.append([i * lattice_constant, j * lattice_constant, k * lattice_constant])  # Corner atoms
                positions.append([i * lattice_constant + lattice_constant / 2, j * lattice_constant + lattice_constant / 2, k * lattice_constant])  # Face-centered atoms
                positions.append([i * lattice_constant + lattice_constant / 2, j * lattice_constant, k * lattice_constant + lattice_constant / 2])
                positions.append([i * lattice_constant, j * lattice_constant + lattice_constant / 2, k * lattice_constant + lattice_constant / 2])
    return np.array(positions)

# Number of unit cells
num_cells = 4
time_steps = np.linspace(0, 2 * np.pi, 100)  # Time steps for animation

# Define wave displacement in different directions
def wave_displacement(position, k, omega, t, wave_direction):
    k_vector = k * np.array(wave_direction) / np.linalg.norm(wave_direction)
    return np.sin(np.dot(k_vector, position) - omega * t)

# Generate the 3D plot and update based on user input
def update_plot(k, omega, lattice_constant, wave_dir):
    fcc_positions = create_fcc_lattice(num_cells, lattice_constant)
    
    # Wave directions
    directions = {
        '[111]': [1, 1, 1],
        '[110]': [1, 1, 0],
        '[100]': [1, 0, 0]
    }
    wave_direction = directions[wave_dir]
    
    # Initial displacement and position update
    t_init = time_steps[0]
    displacements_init = [wave_displacement(pos, k, omega, t_init, wave_direction) for pos in fcc_positions]
    displaced_positions_init = fcc_positions + np.outer(displacements_init, wave_direction / np.linalg.norm(wave_direction))

    # Set up figure for interactive 3D plot
    fig = go.Figure()

    # Plot the lattice with initial displacement
    fig.add_trace(go.Scatter3d(
        x=displaced_positions_init[:, 0],
        y=displaced_positions_init[:, 1],
        z=displaced_positions_init[:, 2],
        mode='markers',
        marker=dict(size=5, color=displacements_init, colorscale='Viridis', opacity=0.8)
    ))

    # Add arrow for wave propagation direction
    arrow_length = 5
    arrow_start = np.array([0, 0, 0])
    arrow_end = np.array(wave_direction) / np.linalg.norm(wave_direction) * arrow_length
    fig.add_trace(go.Scatter3d(
        x=[arrow_start[0], arrow_end[0]],
        y=[arrow_start[1], arrow_end[1]],
        z=[arrow_start[2], arrow_end[2]],
        mode='lines',
        line=dict(color='red', width=5),
        name=f'{wave_dir} Direction'
    ))

    # Add a semi-transparent plane to visualize wave propagation
    plane_length = 6
    vertices = np.array([
        [-plane_length, -plane_length, 0],
        [plane_length, -plane_length, 0],
        [plane_length, plane_length, 0],
        [-plane_length, plane_length, 0]
    ])
    
    # Rotation matrix for aligning the plane with the wave propagation direction
    normal = wave_direction / np.linalg.norm(wave_direction)
    if not np.allclose(normal, [0, 0, 1]):  # Avoid rotation if already aligned with z-axis
        up = np.array([0, 0, 1])
        v = np.cross(up, normal)
        s = np.linalg.norm(v)
        c = np.dot(up, normal)
        vx = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        rotation_matrix = np.eye(3) + vx + np.dot(vx, vx) * ((1 - c) / (s**2))
        rotated_vertices = np.dot(vertices, rotation_matrix.T)
    else:
        rotated_vertices = vertices

    fig.add_trace(go.Mesh3d(
        x=rotated_vertices[:, 0],
        y=rotated_vertices[:, 1],
        z=rotated_vertices[:, 2],
        color='lightblue',
        opacity=0.3,
        name='Wave Propagation Plane'
    ))

    # Create animation frames
    frames = []
    for t in time_steps:
        displacements = [wave_displacement(pos, k, omega, t, wave_direction) for pos in fcc_positions]
        displaced_positions = fcc_positions + np.outer(displacements, wave_direction / np.linalg.norm(wave_direction))
        
        frames.append(go.Frame(data=[go.Scatter3d(
            x=displaced_positions[:, 0],
            y=displaced_positions[:, 1],
            z=displaced_positions[:, 2],
            mode='markers',
            marker=dict(size=5, color=displacements, colorscale='Viridis', opacity=0.8)
        )], name=f"Time = {t:.2f}"))

    # Add animation frames to the figure
    fig.frames = frames

    # Adjusting figure size in the layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        title=f"Wave Propagation in FCC Lattice ({wave_dir})",
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 50, "redraw": True},
                                  "fromcurrent": True, "mode": "immediate"}]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False}, 
                                    "mode": "immediate"}])
            ]
        )],
        sliders=[{
            "pad": {"b": 10},
            "currentvalue": {
                "visible": True,
                "prefix": "Time:",
                "xanchor": "center",
                "font": {"size": 20}
            },
            "steps": [
                {
                    "args": [
                        [t],
                        {"frame": {"duration": 50, "redraw": True}, "mode": "immediate"}
                    ],
                    "label": str(t),
                    "method": "animate",
                } for t in time_steps
            ]
        }],
        autosize=False,        # Disable auto-sizing to set custom dimensions
        width=1200,            # Increase the width of the plot
        height=600             # Increase the height of the plot
    )

    # Show plot
    pio.show(fig)

# Widgets for user input
k_slider = widgets.FloatSlider(value=2 * np.pi, min=0.1, max=10, step=0.1, description='Wavevector (k)')
omega_slider = widgets.FloatSlider(value=2 * np.pi, min=0.1, max=10, step=0.1, description='Frequency (ω)')
a_slider = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1, description='Lattice Constant (a)')
wave_direction_dropdown = widgets.Dropdown(options=['[111]', '[110]', '[100]'], value='[111]', description='Direction')

# Interactive function to update the plot
def interactive_plot(k, omega, lattice_constant, wave_dir):
    update_plot(k, omega, lattice_constant, wave_dir)

# Display interactive sliders and dropdown explicitly
#display(VBox([k_slider, omega_slider, a_slider, wave_direction_dropdown]))
interact(interactive_plot, k=k_slider, omega=omega_slider, lattice_constant=a_slider, wave_dir=wave_direction_dropdown)


interactive(children=(FloatSlider(value=6.283185307179586, description='Wavevector (k)', max=10.0, min=0.1), F…

<function __main__.interactive_plot(k, omega, lattice_constant, wave_dir)>

In [ ]:
### Credits: Vaishnav Sankar K